In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['CK', 'CKs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('creatine kinase', 1271),
 ('cytokinin', 225),
 ('compound k', 120),
 ('cytokeratin', 50),
 ('creatinine kinase', 40),
 ('choline kinase', 31),
 ('carbamate kinase', 21),
 ('cytokines', 21),
 ('casein kinase', 17),
 ('complex karyotype', 14),
 ('cyberknife', 13),
 ('chemokines', 12),
 ('creatine kinase specific activity', 11),
 ('chemokine', 9),
 ('cytokinin trans zeatin riboside', 8),
 ('creatine kinase activity', 7),
 ('creatine kinases', 6),
 ('cathepsin k', 5),
 ('creatinkinase', 5),
 ('cyberknife ®', 5),
 ('such as cytokinins', 5),
 ('chick kidney', 4),
 ('chicken kidney', 4),
 ('changkil', 4),
 ('cyanoketone', 4),
 ('creatinine phosphokinase', 4),
 ('crinkled', 3),
 ('chemokinesis', 3),
 ('cr kinase', 3),
 ('cytokinin content', 3),
 ('calmodulin dependent protein kinase ii', 3),
 ('casein kinases', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-29 04:23:13] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


KeyboardInterrupt: 

In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'calmodulin dependent protein kinase ii': 'HGNC:1697',
  'carbamate kinase': 'MESH:C019288',
  'casein kinase': 'FPLX:CSNK',
  'casein kinases': 'FPLX:CSNK',
  'cathepsin k': 'HGNC:2536',
  'changkil': 'MESH:C440574',
  'chemokine': 'FPLX:Chemokine',
  'chemokines': 'FPLX:Chemokine',
  'chemokinesis': 'GO:GO:0042466',
  'chick kidney': 'ungrounded',
  'chicken kidney': 'ungrounded',
  'choline kinase': 'FPLX:CHK',
  'complex karyotype': 'MESH:D059785',
  'compound k': 'MESH:C112772',
  'cr kinase': 'FPLX:Creatine_kinase',
  'creatine kinase': 'FPLX:Creatine_kinase',
  'creatine kinase activity': 'FPLX:Creatine_kinase',
  'creatine kinase specific activity': 'FPLX:Creatine_kinase',
  'creatine kinases': 'FPLX:Creatine_kinase',
  'creatinine kinase': 'FPLX:Creatine_kinase',
  'creatinine phosphokinase': 'FPLX:Creatine_kinase',
  'creatinkinase': 'FPLX:Creatine_kinase',
  'crinkled': 'ungrounded',
  'cyanoketone': 'MESH:D003489',
  'cyberknife': 'ungrounded',
  'cyberknife ®': 'unground

In [25]:
grounding_map, names, pos_labels = [{'calmodulin dependent protein kinase ii': 'FPLX:CAMK2_complex',
  'carbamate kinase': 'MESH:C019288',
  'casein kinase': 'FPLX:CSNK',
  'casein kinases': 'FPLX:CSNK',
  'cathepsin k': 'HGNC:2536',
  'changkil': 'MESH:C440574',
  'chemokine': 'FPLX:Chemokine',
  'chemokines': 'FPLX:Chemokine',
  'chemokinesis': 'GO:GO:0042466',
  'chick kidney': 'ungrounded',
  'chicken kidney': 'ungrounded',
  'choline kinase': 'FPLX:CHK',
  'complex karyotype': 'MESH:D059785',
  'compound k': 'MESH:C112772',
  'cr kinase': 'FPLX:Creatine_kinase',
  'creatine kinase': 'FPLX:Creatine_kinase',
  'creatine kinase activity': 'FPLX:Creatine_kinase',
  'creatine kinase specific activity': 'FPLX:Creatine_kinase',
  'creatine kinases': 'FPLX:Creatine_kinase',
  'creatinine kinase': 'FPLX:Creatine_kinase',
  'creatinine phosphokinase': 'FPLX:Creatine_kinase',
  'creatinkinase': 'FPLX:Creatine_kinase',
  'crinkled': 'ungrounded',
  'cyanoketone': 'MESH:D003489',
  'cyberknife': 'ungrounded',
  'cyberknife ®': 'ungrounded',
  'cytokeratin': 'MESH:D007633',
  'cytokines': 'MESH:D016207',
  'cytokinin': 'CHEBI:CHEBI:23530',
  'cytokinin content': 'CHEBI:CHEBI:23530',
  'cytokinin trans zeatin riboside': 'CHEBI:CHEBI:71693',
  'such as cytokinins': 'CHEBI:CHEBI:23530'},
 {'FPLX:CAMK2_complex': 'CAMK2_complex',
  'MESH:C019288': 'carbamate kinase',
  'FPLX:CSNK': 'CSNK',
  'HGNC:2536': 'CTSK',
  'MESH:C440574': 'Changkil',
  'FPLX:Chemokine': 'Chemokine',
  'GO:GO:0042466': 'chemokinesis',
  'FPLX:CHK': 'CHK',
  'MESH:D059785': 'Karyotype',
  'MESH:C112772': 'ginsenoside M1',
  'FPLX:Creatine_kinase': 'Creatine_kinase',
  'MESH:D003489': 'Cyanoketone',
  'MESH:D007633': 'Keratins',
  'MESH:D016207': 'Cytokines',
  'CHEBI:CHEBI:23530': 'cytokinin',
  'CHEBI:CHEBI:71693': '9-ribosyl-trans-zeatin'},
 ['FPLX:CHK',
  'FPLX:CSNK',
  'FPLX:Chemokine',
  'FPLX:Creatine_kinase',
  'HGNC:1697',
  'HGNC:2536',
  'MESH:C019288',
  'MESH:C112772',
  'MESH:D007633',
  'MESH:D059785']]

In [26]:
excluded_longforms = []

In [27]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [28]:
additional_entities = {'HGNC:2536': ['CTSK', ['CK']]}

In [29]:
unambiguous_agent_texts = {}

In [30]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [31]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [32]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [33]:
intersection1

[('HGNC:2536', 'HGNC:2536', 480)]

In [16]:
intersection2

[('FPLX:Creatine_kinase', 'HGNC:2536', 0),
 ('MESH:D007633', 'HGNC:2536', 0),
 ('MESH:D016207', 'HGNC:2536', 0),
 ('MESH:C019288', 'HGNC:2536', 0),
 ('MESH:D003489', 'HGNC:2536', 0),
 ('ungrounded', 'HGNC:2536', 0),
 ('FPLX:Chemokine', 'HGNC:2536', 0),
 ('FPLX:CHK', 'HGNC:2536', 0),
 ('FPLX:CSNK', 'HGNC:2536', 0),
 ('MESH:C440574', 'HGNC:2536', 0),
 ('CHEBI:CHEBI:23530', 'HGNC:2536', 0),
 ('MESH:C112772', 'HGNC:2536', 0),
 ('FPLX:CAMK2_complex', 'HGNC:2536', 0),
 ('MESH:D059785', 'HGNC:2536', 0),
 ('HGNC:2536', 'HGNC:2536', 1),
 ('GO:GO:0042466', 'HGNC:2536', 0),
 ('CHEBI:CHEBI:71693', 'HGNC:2536', 0)]

[('HGNC:7321', ['Musculin', 'musculin'])]

In [34]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [35]:
names.update({key: value[0] for key, value in additional_entities.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() | unambiguous_agent_texts.keys())
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})

In [36]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 04:27:22] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 04:27:52] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9456692108337685 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [37]:
classifier.stats

{'label_distribution': {'FPLX:Creatine_kinase': 1028,
  'MESH:D007633': 48,
  'MESH:D016207': 18,
  'MESH:C019288': 11,
  'MESH:D003489': 4,
  'ungrounded': 21,
  'FPLX:Chemokine': 20,
  'FPLX:CHK': 23,
  'FPLX:CSNK': 18,
  'MESH:C440574': 4,
  'CHEBI:CHEBI:23530': 190,
  'MESH:C112772': 91,
  'FPLX:CAMK2_complex': 1,
  'MESH:D059785': 10,
  'HGNC:2536': 5,
  'GO:GO:0042466': 2,
  'CHEBI:CHEBI:71693': 1},
 'f1': {'mean': 0.945669, 'std': 0.007127},
 'precision': {'mean': 0.937941, 'std': 0.008198},
 'recall': {'mean': 0.960905, 'std': 0.006465},
 'FPLX:Chemokine': {'f1': {'mean': 0.788095, 'std': 0.126974},
  'pr': {'mean': 0.7, 'std': 0.187083},
  'rc': {'mean': 0.95, 'std': 0.1}},
 'FPLX:CHK': {'f1': {'mean': 0.926984, 'std': 0.060734},
  'pr': {'mean': 0.87, 'std': 0.107703},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'CHEBI:CHEBI:71693': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0042466': {'f1': {'mean': 0.0, 'std': 0

In [38]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [39]:
disamb.dump(model_name, results_path)

In [40]:
print(disamb.info())

Disambiguation model for CK, and CKs

Produces the disambiguations:
	9-ribosyl-trans-zeatin	CHEBI:CHEBI:71693
	CAMK2_complex	FPLX:CAMK2_complex
	CHK*	FPLX:CHK
	CSNK*	FPLX:CSNK
	CTSK*	HGNC:2536
	Changkil	MESH:C440574
	Chemokine*	FPLX:Chemokine
	Creatine_kinase*	FPLX:Creatine_kinase
	Cyanoketone	MESH:D003489
	Cytokines	MESH:D016207
	Karyotype*	MESH:D059785
	Keratins*	MESH:D007633
	carbamate kinase*	MESH:C019288
	chemokinesis	GO:GO:0042466
	cytokinin	CHEBI:CHEBI:23530
	ginsenoside M1*	MESH:C112772

Class level metrics:
--------------------
Grounding             	Count	F1     
       Creatine_kinase*	1028	0.97116
             cytokinin	 190	0.98139
        ginsenoside M1*	  91	0.87868
              Keratins*	  48	0.88343
                   CHK*	  23	0.92698
            Ungrounded	  21	0.92143
             Chemokine*	  20	 0.7881
             Cytokines	  18	0.26667
                  CSNK*	  18	0.50667
      carbamate kinase*	  11	0.83333
             Karyotype*	  10	    1.0
                

In [41]:
model_to_s3(disamb)